In [ ]:
pip install transformers torch datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [1]:
import pandas as pd
import random
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import os
os.environ["WANDB_DISABLED"] = "true"

#Pruebas (ignorar)


In [ ]:
def generar_cancion(model, tokenizer, input_text, max_length=100):
    inputs = tokenizer.encode(input_text, return_tensors="pt")
    outputs = model.generate(
        inputs,
        max_length=max_length,          # Longitud máxima del texto
        num_return_sequences=1,         # Número de secuencias a generar
        temperature=0.5,                # Aleatoriedad en la generación
        top_k=50,                       # Considera solo los 50 tokens más probables
        top_p=0.9,                      # Nucleus sampling (90% de probabilidad acumulada)
        repetition_penalty=1.5,         # Penaliza repeticiones
        do_sample=True                  # Habilita muestreo en lugar de usar solo los tokens más probables
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

input_text = "Amor eterno"
cancion = generar_cancion(model, tokenizer, input_text, max_length=1000)
print("Canción generada:\n", cancion)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Canción generada:
 Amor eterno
Démigo, no esta que aunque un mejor de tiene (Y) y esperando el mínde en la vida tú lo franca'', pienso podrás por cuenta yo siempre día: ¿quieró?¡Quirte quilez conno todas mi amores te llama'? Ángel noche seguido aquiendo ya sabes las pensos perderon-de razón desarmeñanas - llegado si le escribuente lloramador al cada otro bataña


Chequear que uso la GPU

In [4]:
import torch
print(torch.cuda.is_available())  # Debería devolver True


True


In [5]:
# 1. Cargar el dataset desde Google Drive
ruta_archivo = '/content/drive/MyDrive/NLP/TP final/canciones_artistas.csv'
df = pd.read_csv(ruta_archivo)


# 2. Selección de artistas y porcentajes
def seleccionar_datos(df, distribucion):
    artistas_seleccionados = {k: v for k, v in distribucion.items() if v > 0}
    letras_seleccionadas = []
    for artista, porcentaje in artistas_seleccionados.items():
        letras_artista = df[df['artista'] == artista]['cancion'].tolist()
        n_seleccionadas = int(len(letras_artista) * porcentaje / 100)
        letras_seleccionadas.extend(random.sample(letras_artista, n_seleccionadas))
    return letras_seleccionadas

# Configurar los porcentajes
distribucion = {
    "tini": 0,
    "callejero": 0,
    "spinetta": 100,
    "dillom": 0
}
letras_seleccionadas = seleccionar_datos(df, distribucion)

# Guardar las letras seleccionadas en un archivo de texto
with open("dataset_train.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(letras_seleccionadas))

# 3. Preparar el dataset para el entrenamiento
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="dataset_train.txt",
    block_size=128
)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# 4. Configurar y entrenar el modelo

training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
    report_to="none"  # Desactiva W&B
)


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset
)

trainer.train()


# Mover el modelo a la GPU
model = model.to("cuda")

def generar_cancion(model, tokenizer, input_text, max_length=1000):
    inputs = tokenizer.encode(input_text, return_tensors="pt").to("cuda")
    outputs = model.generate(
        inputs,
        max_length=max_length,          # Longitud máxima del texto
        num_return_sequences=1,         # Número de secuencias a generar
        temperature=0.4,                # Aleatoriedad en la generación
        top_k=100,                       # Considera solo los 50 tokens más probables
        top_p=0.9,                      # Nucleus sampling (90% de probabilidad acumulada)
        repetition_penalty=1.5,         # Penaliza repeticiones
        do_sample=True                  # Habilita muestreo en lugar de usar solo los tokens más probables
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

input_text = "Amor eterno"
cancion = generar_cancion(model, tokenizer, input_text, max_length=1000)
print("Canción generada:\n", cancion)


/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Canción generada:
 Amor eterno
Entre nombres no vez de mi vida, mí y espera en la luz" (Y)

Hoy se habrándote ya te abierto por el mundial del Sol. Donde al fin amor año los bienos que estar tu cielso hastan aquel hagólo sin nunca con las horas otra un parediré despicierdo ("y yo me entendiar") soprás comunidad ademarcín para tus sueñales desde lo capitán atoloso arbol duerma dolor voce durante? Fermes su gran franca farro allende haría ser azul leche sus fuentros voycerme recurvemente volver acabrirando asción verdad apareces tal querer vens encontrarillo solo lluvia son floresque este crepor distancia puestra largipara aleja secura caen sedate rezo bracerla hayendo todalo podesta calligliene cosociado detribero ancon va edificio preguntamentena cuanto corazones altaria nuestro jardin tan colectivitar hojano piercido establira escuchane cruposal andriya diaba panilla galore siudadora canchera descubruza instantedecito finalizionario silviage rientras imprensa maquillojita destino gris

In [6]:
    def generar_cancion(model, tokenizer, input_text, max_length=1000):
        inputs = tokenizer.encode(input_text, return_tensors="pt").to("cuda")
        outputs = model.generate(
            inputs,
            max_length=max_length,          # Longitud máxima del texto
            num_return_sequences=1,         # Número de secuencias a generar
            temperature=0.4,                # Aleatoriedad en la generación
            top_k=100,                       # Considera solo los 50 tokens más probables
            top_p=0.9,                      # Nucleus sampling (90% de probabilidad acumulada)
            repetition_penalty=1.5,         # Penaliza repeticiones
            do_sample=True                  # Habilita muestreo en lugar de usar solo los tokens más probables
        )
        return tokenizer.decode(outputs[0], skip_special_tokens=True)

Aca con los 4 al mismo tiempo

In [ ]:
# 2. Selección de artistas y porcentajes
def seleccionar_datos(df, distribucion):
    artistas_seleccionados = {k: v for k, v in distribucion.items() if v > 0}
    letras_seleccionadas = []
    for artista, porcentaje in artistas_seleccionados.items():
        letras_artista = df[df['artista'] == artista]['cancion'].tolist()
        n_seleccionadas = int(len(letras_artista) * porcentaje / 100)
        letras_seleccionadas.extend(random.sample(letras_artista, n_seleccionadas))
    return letras_seleccionadas

def generar_cancion(model, tokenizer, input_text, max_length=1000):
    inputs = tokenizer.encode(input_text, return_tensors="pt").to("cuda")
    outputs = model.generate(
        inputs,
        max_length=max_length,          # Longitud máxima del texto
        num_return_sequences=1,         # Número de secuencias a generar
        temperature=0.4,                # Aleatoriedad en la generación
        top_k=100,                       # Considera solo los 50 tokens más probables
        top_p=0.9,                      # Nucleus sampling (90% de probabilidad acumulada)
        repetition_penalty=1.5,         # Penaliza repeticiones
        do_sample=True                  # Habilita muestreo en lugar de usar solo los tokens más probables
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:
# 1. Cargar el dataset desde Google Drive
ruta_archivo = '/content/drive/MyDrive/NLP/TP final/canciones_artistas.csv'
df = pd.read_csv(ruta_archivo)

# Lista de artistas para iterar
artistas = ["spinetta", "tini", "callejero", "dillom"]

# Configuración de entrenamiento para cada artista
for artista in artistas:
    print(f"Entrenando modelo con canciones de: {artista}")

    # Seleccionar las canciones del artista actual
    distribucion = {a: 100 if a == artista else 0 for a in artistas}
    letras_seleccionadas = seleccionar_datos(df, distribucion)

    # Guardar las letras seleccionadas en un archivo de texto
    with open("dataset_train.txt", "w", encoding="utf-8") as f:
        f.write("\n".join(letras_seleccionadas))

    # Preparar el dataset para el entrenamiento
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    model = GPT2LMHeadModel.from_pretrained("gpt2")

    train_dataset = TextDataset(
        tokenizer=tokenizer,
        file_path="dataset_train.txt",
        block_size=128
    )
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False
    )

    # Configurar y entrenar el modelo
    training_args = TrainingArguments(
        output_dir=f"./gpt2-finetuned-{artista}",
        overwrite_output_dir=True,
        num_train_epochs=10,
        per_device_train_batch_size=4,
        save_steps=10_000,
        save_total_limit=2,
        prediction_loss_only=True,
        report_to="none"  # Desactiva W&B
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset
    )

    trainer.train()

    # Mover el modelo a la GPU
    model = model.to("cuda")


    # Generar una canción para el artista actual
    print(f"Generando canción para: {artista}")
    input_text = "Amor eterno"
    cancion = generar_cancion(model, tokenizer, input_text, max_length=300)
    print(f"Canción generada para {artista}:\n{cancion}\n")


Entrenando modelo con canciones de: spinetta


/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Generando canción para: spinetta
Canción generada para spinetta:
Amor eterno
Yo si nunca la nena sin soledad, el mundial de sueño. (Ya no es que será) Sol dulce vidé nos volver a las llevas del cielon y acuerdo al fin du paz

Entrenando modelo con canciones de: tini


...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Generando canción para: tini
Canción generada para tini:
Amor eterno
Yo si nunca la nena sin soledad, el mundial de sueño. (Ya no es que será) Sol dulce vidé nos volver a las llevas del cielon y acuerdo al fin du paz

Entrenando modelo con canciones de: callejero


/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss


KeyboardInterrupt: 

In [ ]:
ruta_archivo = '/content/drive/MyDrive/NLP/TP final/canciones_artistas.csv'
df = pd.read_csv(ruta_archivo)


# 2. Selección de artistas y porcentajes
def seleccionar_datos(df, distribucion):
    artistas_seleccionados = {k: v for k, v in distribucion.items() if v > 0}
    letras_seleccionadas = []
    for artista, porcentaje in artistas_seleccionados.items():
        letras_artista = df[df['artista'] == artista]['cancion'].tolist()
        n_seleccionadas = int(len(letras_artista) * porcentaje / 100)
        letras_seleccionadas.extend(random.sample(letras_artista, n_seleccionadas))
    return letras_seleccionadas

# Configurar los porcentajes
distribucion = {
    "tini": 100,
    "callejero": 0,
    "spinetta": 0,
    "dillom": 0
}
letras_seleccionadas = seleccionar_datos(df, distribucion)

# Guardar las letras seleccionadas en un archivo de texto
with open("dataset_train.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(letras_seleccionadas))

# 3. Preparar el dataset para el entrenamiento
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="dataset_train.txt",
    block_size=128
)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# 4. Configurar y entrenar el modelo

training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
    report_to="none"  # Desactiva W&B
)


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset
)

trainer.train()


# Mover el modelo a la GPU
model = model.to("cuda")

def generar_cancion(model, tokenizer, input_text, max_length=1000):
    inputs = tokenizer.encode(input_text, return_tensors="pt").to("cuda")
    outputs = model.generate(
        inputs,
        max_length=max_length,          # Longitud máxima del texto
        num_return_sequences=1,         # Número de secuencias a generar
        temperature=0.4,                # Aleatoriedad en la generación
        top_k=100,                       # Considera solo los 50 tokens más probables
        top_p=0.9,                      # Nucleus sampling (90% de probabilidad acumulada)
        repetition_penalty=1.5,         # Penaliza repeticiones
        do_sample=True                  # Habilita muestreo en lugar de usar solo los tokens más probables
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

input_text = "Amor eterno"
cancion = generar_cancion(model, tokenizer, input_text, max_length=1000)
print("Canción generada:\n", cancion)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Canción generada:
 Amor eterno
Entiendes en el mundial de vivir la espana, saldará del suerte. (Ya no es un hombre)

¿Que estoye al fin? Sol que me niño donde amanecuerdo nena hasten aquí por cielos ya tambien a las horas y nunca cosando con tu soledad! Hasta más llegadas se aparecido yo ojate comunidas atalán ademarcín para lo encontrarólo sin vez venida todos los guerrones... Dios enticidades despobrirément asociado destino allende mi podría verdes cuatro son quierrantano te arrepentire este boca acaba azulce blancoso securosario hoque aleja desde crecerno solo volvermejo). Nunquie escribara caesi hablor duviendo pieperidad cruces harabazón durante leche nuestra puistén puedome diagrafia gran abandango anelillo ser tempracio talaga panza distionale il fermito frutigo sublanta rayonzo callejas reba manojiste canciarte llorajo tan rajatira bracerla finalizada.) Fuego fue sueñosa era tenera salve si pecurritol establis silencio tiene sus peligros flores da diamantes oreste piernas mirad

# Generando canciones de cada artista



In [3]:
# 1. Cargar el dataset desde Google Drive
ruta_archivo = '/content/drive/MyDrive/NLP/TP final/canciones_artistas.csv'
df = pd.read_csv(ruta_archivo)


# 2. Selección de artistas y porcentajes
def seleccionar_datos(df, distribucion):
    artistas_seleccionados = {k: v for k, v in distribucion.items() if v > 0}
    letras_seleccionadas = []
    for artista, porcentaje in artistas_seleccionados.items():
        letras_artista = df[df['artista'] == artista]['cancion'].tolist()
        n_seleccionadas = int(len(letras_artista) * porcentaje / 100)
        letras_seleccionadas.extend(random.sample(letras_artista, n_seleccionadas))
    return letras_seleccionadas



In [ ]:
df[df['artista'] == artista]['cancion']

## Spinetta

In [4]:
def generar_cancion_spinetta(model, tokenizer, input_text, max_length=1000):
    inputs = tokenizer.encode(input_text, return_tensors="pt").to("cuda")
    outputs = model.generate(
        inputs,
        max_length=max_length,          # Longitud máxima del texto
        num_return_sequences=1,         # Número de secuencias a generar
        temperature=0.4,                # Aleatoriedad en la generación
        top_k=100,                       # Considera solo los 50 tokens más probables
        top_p=0.9,                      # Nucleus sampling (90% de probabilidad acumulada)
        repetition_penalty=1.5,         # Penaliza repeticiones
        do_sample=True                  # Habilita muestreo en lugar de usar solo los tokens más probables
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [5]:
# Configurar los porcentajes
distribucion = {
    "tini": 0,
    "callejero": 0,
    "spinetta": 100,
    "dillom": 0
}
letras_seleccionadas_spinetta = seleccionar_datos(df, distribucion)

# Guardar las letras seleccionadas en un archivo de texto
with open("dataset_train_spinetta.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(letras_seleccionadas_spinetta))

# 3. Preparar el dataset para el entrenamiento
tokenizer_spinetta = GPT2Tokenizer.from_pretrained("gpt2")
model_spinetta = GPT2LMHeadModel.from_pretrained("gpt2")

train_dataset_spinetta = TextDataset(
    tokenizer=tokenizer_spinetta,
    file_path="dataset_train_spinetta.txt",
    block_size=128
)
data_collator_spinetta = DataCollatorForLanguageModeling(
    tokenizer=tokenizer_spinetta,
    mlm=False
)

# 4. Configurar y entrenar el modelo

training_args_spinetta = TrainingArguments(
    output_dir="./gpt2-finetuned_spinetta",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=4,
    save_steps=10_000,
    learning_rate=0.001,
    save_total_limit=2,
    prediction_loss_only=True,
    report_to="none"  # Desactiva W&B
)

trainer_spinetta = Trainer(
    model=model_spinetta,
    args=training_args_spinetta,
    data_collator=data_collator_spinetta,
    train_dataset=train_dataset_spinetta
)

trainer_spinetta.train()


# Mover el modelo a la GPU
model_spinetta = model_spinetta.to("cuda")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss


In [6]:
input_text = "Amor eterno"
cancion = generar_cancion_spinetta(model_spinetta, tokenizer_spinetta, input_text, max_length=200)
print("Canción generada:\n", cancion)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Canción generada:
 Amor eterno el sol
y un guerrero no detiene jamás
su marcha. Fermín se fue a la vida
No corras más
Tu tiempo es hoy
Y al fin mi duende nació conservar y cuidar con amor este jardín de gente
Eterno en el día sin Sol
Algún arbol que robé cuidará su boca saldrá
Más si tu cielo brisa
Cómo haré para ver así ya volver
Que ni los sueños living ojaron de llover
Sube, saltan por estante

Pretenda volviendo canción recirculemente idiota voz le confronto ha de la galaxia

La luz
Nunca callés ser felicen sus manes respondrir


## Tini

In [7]:
# Configurar los porcentajes
distribucion = {
    "tini": 100,
    "callejero": 0,
    "spinetta": 0,
    "dillom": 0
}
letras_seleccionadas_tini = seleccionar_datos(df, distribucion)

# Guardar las letras seleccionadas en un archivo de texto
with open("dataset_train_tini.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(letras_seleccionadas_tini))

# 3. Preparar el dataset para el entrenamiento
tokenizer_tini = GPT2Tokenizer.from_pretrained("gpt2")
model_tini = GPT2LMHeadModel.from_pretrained("gpt2")

train_dataset_tini = TextDataset(
    tokenizer=tokenizer_tini,
    file_path="dataset_train_tini.txt",
    block_size=128
)
data_collator_tini = DataCollatorForLanguageModeling(
    tokenizer=tokenizer_tini,
    mlm=False
)

# 4. Configurar y entrenar el modelo

training_args_tini = TrainingArguments(
    output_dir="./gpt2-finetuned_tini",
    overwrite_output_dir=True,
    num_train_epochs=25,
    per_device_train_batch_size=4,
    learning_rate=0.001,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
    report_to="none"  # Desactiva W&B
)


trainer_tini = Trainer(
    model=model_tini,
    args=training_args_tini,
    data_collator=data_collator_tini,
    train_dataset=train_dataset_tini
)

trainer_tini.train()


# Mover el modelo a la GPU
model_tini = model_tini.to("cuda")


/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss
500,1.223000
1000,0.044600


In [ ]:
def generar_cancion_tini(model, tokenizer, input_text, max_length=1000):
    inputs = tokenizer.encode(input_text, return_tensors="pt").to("cuda")
    outputs = model.generate(
        inputs,
        max_length=max_length,          # Longitud máxima del texto
        num_return_sequences=1,         # Número de secuencias a generar
        temperature=0.5,                # Aleatoriedad en la generación
        top_k=50,                       # Considera solo los 50 tokens más probables
        top_p=0.8,                      # Nucleus sampling (90% de probabilidad acumulada)
        repetition_penalty=1.5,         # Penaliza repeticiones
        do_sample=True                  # Habilita muestreo en lugar de usar solo los tokens más probables
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
tokenizer_tini.pad_token = tokenizer_tini.eos_token
def generar_cancion_tini(model, tokenizer, input_text, max_length=1000):
    # Tokenizar el texto de entrada con atención y relleno
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        padding=True,              # Asegura que haya relleno si es necesario
        truncation=True,           # Trunca el texto si excede el tamaño máximo
        max_length=128             # Tamaño máximo del bloque
    )
    inputs = {key: value.to("cuda") for key, value in inputs.items()}  # Mover a GPU

    # Generar texto
    outputs = model.generate(
        **inputs,                  # Incluye input_ids y attention_mask
        max_length=max_length,     # Longitud máxima del texto
        num_return_sequences=1,    # Número de secuencias a generar
        temperature=0.5,           # Aleatoriedad en la generación
        top_k=100,                  # Considera solo los 50 tokens más probables
        top_p=0.9,                 # Nucleus sampling (90% de probabilidad acumulada)
        repetition_penalty=1.5,    # Penaliza repeticiones
        do_sample=True             # Habilita muestreo en lugar de usar solo los tokens más probables
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:
input_text = "Amor eterno"
cancion = generar_cancion_tini(model_tini, tokenizer_tini, input_text, max_length=600)
print("Canción generada:\n", cancion)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Canción generada:
 Amor eterno

Porque ahora lo que soy
Me encanta este desaire
Este es mi último baile (uh)

A veces siento que caigo y que vivo importa
Ya no se enamora, más fuerte
Hasta TINI, TINI esta noche yo soy
Dos mil thirty five
That's right ya no te puede ver
Baby: Baby, yo no te vayas
Tini, no te vayas
Qué te vayas
Los dejé palabras pa' antes, no te vayas

Cambiando el Soltará
Así en la momento, y tomoëtú neces tús un beso adió
Que ya noSi me hace a traició
Un viatro¿Por
Siembre
Ni de tiene y siе extrañas
Ay, y sigasa
Ah
No saben las despе miedo y ya sabaz
Nena
(¡ngelas
Oga, boluda, boluda

Vive yo solo invierda fue mi costa rumboemos a tu corazón y ya no te vida
Se hombre hasta 90
Onlegan
Pa' dentro') ¿)</iciste y ahora perdería a travame un trago

Estoy a traventura late
Guardaré sus mar
Corazón trauе invoca
Tuvo a ti, trave mi culpa y ahora nos fresa y ahora pa' alimentan contratar
(y yo aquí ya peque sabia
Algo ya no venir mi mina traovió ya sufridave contra la persiana 

##Dillom


In [ ]:
def generar_cancion_dillom(model, tokenizer, input_text, max_length=1000):
    inputs = tokenizer.encode(input_text, return_tensors="pt").to("cuda")
    outputs = model.generate(
        inputs,
        max_length=max_length,          # Longitud máxima del texto
        num_return_sequences=1,         # Número de secuencias a generar
        temperature=0.5,                # Aleatoriedad en la generación
        top_k=50,                       # Considera solo los 50 tokens más probables
        top_p=0.9,                      # Nucleus sampling (90% de probabilidad acumulada)
        repetition_penalty=1.5,         # Penaliza repeticiones
        do_sample=True                  # Habilita muestreo en lugar de usar solo los tokens más probables
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:

# Configurar los porcentajes
distribucion = {
    "tini": 0,
    "callejero": 0,
    "spinetta": 0,
    "dillom": 100
}
letras_seleccionadas_dillom = seleccionar_datos(df, distribucion)

# Guardar las letras seleccionadas en un archivo de texto
with open("dataset_train_dillom.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(letras_seleccionadas_dillom))

# 3. Preparar el dataset para el entrenamiento
tokenizer_dillom = GPT2Tokenizer.from_pretrained("gpt2")
model_dillom = GPT2LMHeadModel.from_pretrained("gpt2")

train_dataset_dillom = TextDataset(
    tokenizer=tokenizer_dillom,
    file_path="dataset_train_dillom.txt",
    block_size=128
)
data_collator_dillom = DataCollatorForLanguageModeling(
    tokenizer=tokenizer_dillom,
    mlm=False
)

# 4. Configurar y entrenar el modelo

training_args_dillom = TrainingArguments(
    output_dir="./gpt2-finetuned_dillom",
    overwrite_output_dir=True,
    num_train_epochs=20,
    learning_rate=0.0005,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
    report_to="none"  # Desactiva W&B
)


trainer_dillom = Trainer(
    model=model_dillom,
    args=training_args_dillom,
    data_collator=data_collator_dillom,
    train_dataset=train_dataset_dillom
)

trainer_dillom.train()


# Mover el modelo a la GPU
model_dillom = model_dillom.to("cuda")



/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss
500,1.153300


In [8]:
input_text = "Amor eterno"
cancion = generar_cancion_dillom(model_dillom, tokenizer_dillom, input_text, max_length=200)
print("Canción generada:\n", cancion)

NameError: name 'generar_cancion_dillom' is not defined

## Callejero

In [ ]:
def generar_cancion_callejero(model, tokenizer, input_text, max_length=1000):
    inputs = tokenizer.encode(input_text, return_tensors="pt").to("cuda")
    outputs = model.generate(
        inputs,
        max_length=max_length,          # Longitud máxima del texto
        num_return_sequences=1,         # Número de secuencias a generar
        temperature=0.5,                # Aleatoriedad en la generación
        top_k=50,                       # Considera solo los 50 tokens más probables
        top_p=0.9,                      # Nucleus sampling (90% de probabilidad acumulada)
        repetition_penalty=1.5,         # Penaliza repeticiones
        do_sample=True                  # Habilita muestreo en lugar de usar solo los tokens más probables
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:

# Configurar los porcentajes
distribucion = {
    "tini": 0,
    "callejero": 100,
    "spinetta": 0,
    "dillom": 0
}
letras_seleccionadas_callejero = seleccionar_datos(df, distribucion)

# Guardar las letras seleccionadas en un archivo de texto
with open("dataset_train_callejero.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(letras_seleccionadas_callejero))

# 3. Preparar el dataset para el entrenamiento
tokenizer_callejero = GPT2Tokenizer.from_pretrained("gpt2")
model_callejero = GPT2LMHeadModel.from_pretrained("gpt2")

train_dataset_callejero = TextDataset(
    tokenizer=tokenizer_callejero,
    file_path="dataset_train_callejero.txt",
    block_size=128
)
data_collator_callejero = DataCollatorForLanguageModeling(
    tokenizer=tokenizer_callejero,
    mlm=False
)

# 4. Configurar y entrenar el modelo

training_args_callejero = TrainingArguments(
    output_dir="./gpt2-finetuned_callejero",
    overwrite_output_dir=True,
    num_train_epochs=15,
    learning_rate=0.0005,
    per_device_train_batch_size=4,
    save_steps=500,
    save_total_limit=2,
    prediction_loss_only=True,
    report_to="none"  # Desactiva W&B
)


trainer_callejero = Trainer(
    model=model_callejero,
    args=training_args_callejero,
    data_collator=data_collator_callejero,
    train_dataset=train_dataset_callejero
)

trainer_callejero.train()


# Mover el modelo a la GPU
model_callejero = model_callejero.to("cuda")



/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss
500,1.574500


In [ ]:
input_text = "Amor eterno"
cancion = generar_cancion_callejero(model_callejero, tokenizer_callejero, input_text, max_length=200)
print("Canción generada:\n", cancion)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Canción generada:
 Amor eterno y la más cara
Nosotros somos los mejores amigos
En mi podés confiar
No me vengas nada, yo lo que hoy se te re gustar
Pero no tienen febrero, por ese 'tán celoso' (na-na) A tu gato ya ni me pongo en el clínico
Lo' gato' que toqué así está bellaca pa'l quedar
Ella sabe que tiene alta burra conmigo acuerdo
Si nos miran, entonce' no me voy rally de espionaje', ah-ah
¿Qué? Ah! Alan Gome’
Hagan casi past tenemos día anda la resaca en sueltas
Yo soy milipili un bueno como cómo jue


## Callejero + Tini

In [43]:
def generar_cancion_tini_callejero(model, tokenizer, input_text, max_length=1000):
    inputs = tokenizer.encode(input_text, return_tensors="pt").to("cuda")
    outputs = model.generate(
        inputs,
        max_length=max_length,          # Longitud máxima del texto
        num_return_sequences=1,         # Número de secuencias a generar
        temperature=0.5,                # Aleatoriedad en la generación
        top_k=50,                       # Considera solo los 50 tokens más probables
        top_p=0.9,                      # Nucleus sampling (90% de probabilidad acumulada)
        repetition_penalty=1.5,         # Penaliza repeticiones
        do_sample=True                  # Habilita muestreo en lugar de usar solo los tokens más probables
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [44]:
# Configurar los porcentajes
distribucion = {
    "tini": 100,
    "callejero": 80,
    "spinetta": 0,
    "dillom": 0
}
letras_seleccionadas_tini_callejero = seleccionar_datos(df, distribucion)

# Guardar las letras seleccionadas en un archivo de texto
with open("dataset_train_tini_callejero.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(letras_seleccionadas_tini_callejero))

# 3. Preparar el dataset para el entrenamiento
tokenizer_tini_callejero = GPT2Tokenizer.from_pretrained("gpt2")
model_tini_callejero = GPT2LMHeadModel.from_pretrained("gpt2")

train_dataset_tini_callejero = TextDataset(
    tokenizer=tokenizer_tini_callejero,
    file_path="dataset_train_tini_callejero.txt",
    block_size=128
)
data_collator_tini_callejero = DataCollatorForLanguageModeling(
    tokenizer=tokenizer_tini_callejero,
    mlm=False
)

# 4. Configurar y entrenar el modelo

training_args_tini_callejero = TrainingArguments(
    output_dir="./gpt2-finetuned_tini_callejero",
    overwrite_output_dir=True,
    num_train_epochs=10,
    learning_rate=0.001,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
    report_to="none"  # Desactiva W&B
)


trainer_tini_callejero = Trainer(
    model=model_tini_callejero,
    args=training_args_tini_callejero,
    data_collator=data_collator_tini_callejero,
    train_dataset=train_dataset_tini_callejero
)

trainer_tini_callejero.train()


# Mover el modelo a la GPU
model_tini_callejero = model_tini_callejero.to("cuda")



/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss
500,2.345000


In [50]:
input_text = "Amor eterno"
cancion = generar_cancion_tini_callejero(model_tini_callejero, tokenizer_tini_callejero, input_text, max_length=500)
print("Canción generada:\n", cancion)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Canción generada:
 Amor eterno, uoh
Libre soy, libre soy
Libertad sin vuelta atrás

Y firme así
Me quedó aquí
No me a ser yo aquí
Fua, mi amor
Pero solo hay una vida de tu corazón

Una vida de frícil (uh)

Debo estar loca, no poder salir
Nadie sabe que es maldita
Hay lo mismo cuando estés triste pensándome en la cama
Con su cari de ti te vas a estar con algo
No pienso volver el fue tiempo

Una vez dice polisé haces hicen parengas más y diga en lo otra hasta


Cuando ya segú parengas se encuvia dejes se encender por quierda hermana
Sorry, perderte ahora que son hería


Y no seas el se encima
Si nosotros secuvia se fue ya se fue hermanos
Sabe' en las demás
Pensando arde las demostrar
Pensimos polaridad
Para decís dame en más
Seguridad acuerdo que el problema sol sub her fin te arribí
Algo dejes verdad se enc��nas seguir

Tres que encontrás

Y si son hermanos seguias por miento que son heridas por los guerí

Pude hablan pasamos
Y yo nunca arma
Son hermela

Estoy son heridas caver quién pa

## Spinetta + Dillom

In [51]:
def generar_cancion_sd(model, tokenizer, input_text, max_length=1000):
    inputs = tokenizer.encode(input_text, return_tensors="pt").to("cuda")
    outputs = model.generate(
        inputs,
        max_length=max_length,          # Longitud máxima del texto
        num_return_sequences=1,         # Número de secuencias a generar
        temperature=0.5,                # Aleatoriedad en la generación
        top_k=50,                       # Considera solo los 50 tokens más probables
        top_p=0.9,                      # Nucleus sampling (90% de probabilidad acumulada)
        repetition_penalty=1.5,         # Penaliza repeticiones
        do_sample=True                  # Habilita muestreo en lugar de usar solo los tokens más probables
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [52]:
# Configurar los porcentajes
distribucion = {
    "tini": 0,
    "callejero": 0,
    "spinetta": 100,
    "dillom": 50
}
letras_seleccionadas_sd = seleccionar_datos(df, distribucion)

# Guardar las letras seleccionadas en un archivo de texto
with open("dataset_train_sd.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(letras_seleccionadas_sd))

# 3. Preparar el dataset para el entrenamiento
tokenizer_sd = GPT2Tokenizer.from_pretrained("gpt2")
model_sd = GPT2LMHeadModel.from_pretrained("gpt2")

train_dataset_sd = TextDataset(
    tokenizer=tokenizer_sd,
    file_path="dataset_train_sd.txt",
    block_size=128
)
data_collator_sd = DataCollatorForLanguageModeling(
    tokenizer=tokenizer_sd,
    mlm=False
)

# 4. Configurar y entrenar el modelo

training_args_sd = TrainingArguments(
    output_dir="./gpt2-finetuned_sd",
    overwrite_output_dir=True,
    num_train_epochs=10,
    learning_rate=0.001,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
    report_to="none"  # Desactiva W&B
)


trainer_sd = Trainer(
    model=model_sd,
    args=training_args_sd,
    data_collator=data_collator_sd,
    train_dataset=train_dataset_sd
)

trainer_sd.train()


# Mover el modelo a la GPU
model_sd = model_sd.to("cuda")

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss
500,1.811500


In [66]:
input_text = "Amor eterno"
cancion = generar_cancion_sd(model_sd, tokenizer_sd, input_text, max_length=500)
print("Canción generada:\n", cancion)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Canción generada:
 Amor eterno y sin ir al dentista
Suena un poco a popper con el jarabe, como si me pregunta: Dillom (what?)
Huh?
Oh (oh)
Huh (eh), puta, cojamo' en el trunk
Ahora no jodas like Kronk
Lo fumamo' en pipa y en bong

Puta, cojo gana plata pelotuda del Chaco
Buscando la saco cuiden
Ahora ni mirarte es Balención
Estoy haciendo mucha raya de Avril Lavigne pero y me esperan a mí guita que yo soy le da
Me pelearía pa’ que le confiario más pillo ahora está rezan paixao CBDAYRey (holiеro—

Los mandamos fucked upan aire el culo pa’ que DMT (ah)



Registral, mediodas chico sieto fuck around this city
Fuck that bitchesicletad

Yo tengo us vayamos coca mi triste (bueno y Claptoniones hallaba ropa por fa, como en Centamario opa reguentó shady times
Coserpechado y Claví lili me voy a vez milajesito y naysí cajes desde ser el asterijo fuck Holy
Sobreble G20, mm
Ella me divién
Busqué la tapa parec y no
No, comienen y tanto que no me shawty quiero clogger listo muy sexy

Tampoco a wacha

##Spinetta+Callejero

In [67]:
def generar_cancion_sc(model, tokenizer, input_text, max_length=1000):
    inputs = tokenizer.encode(input_text, return_tensors="pt").to("cuda")
    outputs = model.generate(
        inputs,
        max_length=max_length,          # Longitud máxima del texto
        num_return_sequences=1,         # Número de secuencias a generar
        temperature=0.5,                # Aleatoriedad en la generación
        top_k=50,                       # Considera solo los 50 tokens más probables
        top_p=0.9,                      # Nucleus sampling (90% de probabilidad acumulada)
        repetition_penalty=1.5,         # Penaliza repeticiones
        do_sample=True                  # Habilita muestreo en lugar de usar solo los tokens más probables
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [77]:
# Configurar los porcentajes
distribucion = {
    "tini": 0,
    "callejero": 10,
    "spinetta": 100,
    "dillom": 0
}
letras_seleccionadas_sc = seleccionar_datos(df, distribucion)

# Guardar las letras seleccionadas en un archivo de texto
with open("dataset_train_sc.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(letras_seleccionadas_sc))

# 3. Preparar el dataset para el entrenamiento
tokenizer_sc = GPT2Tokenizer.from_pretrained("gpt2")
model_sc = GPT2LMHeadModel.from_pretrained("gpt2")

train_dataset_sc = TextDataset(
    tokenizer=tokenizer_sc,
    file_path="dataset_train_sc.txt",
    block_size=128
)
data_collator_sc = DataCollatorForLanguageModeling(
    tokenizer=tokenizer_sc,
    mlm=False
)

# 4. Configurar y entrenar el modelo

training_args_sc = TrainingArguments(
    output_dir="./gpt2-finetuned_sc",
    overwrite_output_dir=True,
    num_train_epochs=10,
    learning_rate=0.001,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
    report_to="none"  # Desactiva W&B
)


trainer_sc = Trainer(
    model=model_sc,
    args=training_args_sc,
    data_collator=data_collator_sc,
    train_dataset=train_dataset_sc
)

trainer_sc.train()


# Mover el modelo a la GPU
model_sc = model_sc.to("cuda")

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss
500,1.728800


In [84]:
input_text = "Amor eterno"
cancion = generar_cancion_sc(model_sc, tokenizer_sc, input_text, max_length=500)
print("Canción generada:\n", cancion)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Canción generada:
 Amor eterno
Y así verás lo bueno y dulce que es amar. Atentamente
La Banda de los Millones, sos re gilidad

Yo a tu corazón sola su periplo
Su equipo es tan precario como un cielo

En el Titán
Pa' que hagamos el plan
De darnos blam-blan
Y llevarte al beat todos gritan juegue desde vivir.

Alguien debió conservar con amor este jardín sin mostrarle
ya música para rompejo tal vez qué será
Que te gusta la roban
Lo río será frío será
y en la mar evitarme un ponerte
y triste estoy solo
ya dormido
ya canción banderín se destruicio del destruicio
la narcaremente hasta secuante podrínate allen el abrio
estrarle tendrás flor cancien aquí...na jarra de huele
 color
frutesenta milipiliatín
dale gracios
jalen panes que hueco y pelear
�ngel que robinado que guerba
yo despotro no detenga
ver
illos salís de pan
és que pan
yehasta que podemos de mi wachín
ge venime mis
ya que hoy nacería olvidarlen y arroque sié
ys
este despierto sonando feliz
Sin brisa hojarasca
Relojera de paso tie

##